<a href="https://colab.research.google.com/github/AbdallahZerfaoui/Project-3---Application-for-Public-Health-Service/blob/main/analyse_finbox_response.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import re
from collections import Counter
from statistics import mean

In [ ]:
class CellRange:
    """Utility class for cell and range operations."""

    @staticmethod
    def col_to_num(col_str):
        num = 0
        for c in col_str.upper():
            num = num * 26 + (ord(c) - ord('A')) + 1
        return num

    @staticmethod
    def num_to_col(col_num):
        string = ""
        while col_num > 0:
            col_num, remainder = divmod(col_num - 1, 26)
            string = chr(65 + remainder) + string
        return string

    @staticmethod
    def get_cell_id(col_num, row_num):
        return f"{CellRange.num_to_col(col_num)}{row_num}"

    @staticmethod
    def parse_range(range_str):
        parts = range_str.split(':')
        if len(parts) != 2:
            return None
        start, end = parts
        start_match = re.match(r'([A-Z]+)(\d+)', start.upper())
        end_match = re.match(r'([A-Z]+)(\d+)', end.upper())
        if not start_match or not end_match:
            return None
        sc, sr = start_match.groups()
        ec, er = end_match.groups()
        try:
            sc_num = CellRange.col_to_num(sc)
            ec_num = CellRange.col_to_num(ec)
            sr_num, er_num = int(sr), int(er)
            sr_num, er_num = sorted((sr_num, er_num))
            sc_num, ec_num = sorted((sc_num, ec_num))
            return (sc_num, sr_num, ec_num, er_num)
        except ValueError:
            return None

    @staticmethod
    def is_numerical(value):
        return isinstance(value, (int, float))

In [ ]:
class Workbook:
    """Represents the entire workbook with all sheets."""

    def __init__(self, file_path):
        self.sheets = {}
        self.load_from_file(file_path)

    def load_from_file(self, file_path):
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
        except FileNotFoundError:
            raise ValueError(f"File not found: {file_path}")
        except json.JSONDecodeError:
            raise ValueError("Invalid JSON format")

        sheets_data = data.get("data", {}).get("model", {}).get("workbook", {}).get("sheets", {})
        for sheet_name, sheet_data in sheets_data.items():
            self.sheets[sheet_name] = Sheet(sheet_name, sheet_data)

    def get_sheet(self, name):
        return self.sheets.get(name)

    def get_all_section_values(self):
        all_sections = {}
        for sheet in self.sheets.values():
            all_sections.update(sheet.process_sections())
        return all_sections



In [ ]:
class Sheet:
    """Represents a single sheet within the workbook."""

    def __init__(self, name, sheet_data):
        self.name = name
        self.cells = sheet_data.get("cells", {})

    def sort_dict_by_key(self, d):
        sorted_d = dict(sorted(d.items(), key=lambda x: str(x[0])))
        return sorted_d

    def sort_dict_by_value(self, d):
        sorted_d = dict(sorted(d.items(), key=lambda x: str(x[1])))
        return sorted_d

    def process_sections(self):
        section_cells = set()
        for cell_id, cell_props in self.cells.items():
            section_info = cell_props.get("section")
            if section_info:
                parts = section_info.split(';')
                if len(parts) == 2:
                    parsed = CellRange.parse_range(parts[1])
                    if parsed:
                        sc, sr, ec, er = parsed
                        for r in range(sr, er + 1):
                            for c in range(sc, ec + 1):
                                section_cells.add(CellRange.get_cell_id(c, r))

        section_values = {}
        for cell_id in section_cells:
            cell = self.cells.get(cell_id)
            if cell and CellRange.is_numerical(cell.get("value")):
                section_values[cell_id] = cell["value"]
        return section_values

    def get_data_range(self, row, start_col, end_col):
        sc = CellRange.col_to_num(start_col)
        ec = CellRange.col_to_num(end_col)
        data = {}
        for col in range(sc, ec + 1):
            cell_id = CellRange.get_cell_id(col, row)
            cell = self.cells.get(cell_id)
            if cell and CellRange.is_numerical(cell.get("value")):
                data[col] = cell["value"]
            else:
                data[col] = 0

        return data

    def get_projection(self, row, start_col, end_col, start_year):
        sc = CellRange.col_to_num(start_col)
        ec = CellRange.col_to_num(end_col)
        projections = {}
        for col in range(sc, ec + 1):
            cell_id = CellRange.get_cell_id(col, row)
            cell = self.cells.get(cell_id)
            if cell and CellRange.is_numerical(cell.get("value")):
                year = start_year + (col - sc)
                projections[year] = cell["value"]
            else:
                year = start_year + (col - sc)
                projections[year] = 0
        return projections

    def get_cell_value(self, cell_id):
        return self.cells.get(cell_id, {}).get("value")

    def get_quater_nwc_values(self):
        total_current_assets = self.get_data_range(71, 'D', 'I')
        cash_and_short_term_invest = self.get_data_range(72, 'D', 'I')
        total_current_liab = self.get_data_range(73, 'D','I')

        current_portion_of_debt = Counter(self.get_data_range(298, 'D', 'I')) + \
          Counter(self.get_data_range(299, 'D', 'I')) + \
          Counter(self.get_data_range(300, 'D', 'I'))

        net_working_capital = Counter(total_current_assets) + \
          Counter(cash_and_short_term_invest) + \
          Counter(total_current_liab) + \
          Counter(current_portion_of_debt)

        return current_portion_of_debt

    def get_nwc_ratios(self):
        net_working_capital = list(self.get_quater_nwc_values().values())
        quater_revenue = self.get_data_range(289, 'D', 'J')
        quater_revenue = list(quater_revenue.values())
        revenues = [sum(quater_revenue[i:i+4]) for i in range(0, len(quater_revenue) - 3)]
        ratios = [nwc/revenue for nwc, revenue in zip(net_working_capital[-4:], revenues)]
        return ratios

    def get_nwc_avg_ratio(self):
        ratios = self.get_nwc_ratios()
        return mean(ratios)

    def get_yearly_nwc_values(self):
        revenues = {}
        revenues = self.get_projection(377, 'E', 'N', 2025)
        revenues["2024"] = self.get_cell_value("H253")
        revenues["2023"] = self.get_cell_value("G253")
        avg_nwc_ratio = self.get_nwc_avg_ratio()
        nwc_values = {year: value * avg_nwc_ratio for year, value in revenues.items()}
        nwc_values = self.sort_dict_by_key(nwc_values)
        return nwc_values

    def get_nwc_investment(self):
        """
        nwc_investment for year n is positive if the nwc_capital decrease compared to n - 1
        nwc_investment for year n is negative if the nwc_capital increase compared to n - 1
        """
        nwc_values = self.get_yearly_nwc_values()
        nwc_investment = {}
        keys = list(nwc_values.keys())
        values = list(nwc_values.values())
        for i in range(1, len(keys)):
            nwc_investment[keys[i]] = values[i - 1] - values[i]
        return nwc_investment




In [ ]:
# Usage Example
if __name__ == "__main__":
    # Load workbook and process sections
    workbook = Workbook("response_MCD.json")
    # result_dict = workbook.get_all_section_values()
    # print("Section Cells:", json.dumps(result_dict, indent=2))

    # Extract EBIT projections
    sheet = workbook.get_sheet("10 Year DCF - Growth Exit")
    if sheet:
        ebit = sheet.get_projection(381, 'E', 'N', 2025)
        print("EBIT Projections:", json.dumps(ebit, indent=2))
        DA = sheet.get_projection(379, 'E', 'N', 2025)
        print("D&A Projections:", json.dumps(DA, indent=2))
        tax_rate = sheet.get_cell_value('D111')
        print("Tax Rate:", tax_rate)

        # pro_format_taxes = ebit
        pro_format_taxes = {year: value * tax_rate for year, value in ebit.items()}
        print("Projected Format Taxes:", json.dumps(pro_format_taxes, indent=2))

        no_pat = Counter(ebit) - Counter(pro_format_taxes)
        print("No PAT:", json.dumps(no_pat, indent=2))

        capital_expanditure = sheet.get_projection(382, 'E', 'N', 2025)
        print("Capital Expanditure Projections:", json.dumps(capital_expanditure, indent=2))

        nwc_quater_values = sheet.get_quater_nwc_values()
        print("NWC Quater Values:", json.dumps(nwc_quater_values, indent=2))

        ratios = sheet.get_nwc_ratios()
        print("NWC Ratios:", ratios)
        print("agv:", mean(ratios))

        nwc_investment = sheet.get_nwc_investment()
        print("NWC Investment:", json.dumps(nwc_investment, indent=2))
        # nwc_vales = sheet.get_nwc_values()
        # print("NWC Values:", json.dumps(nwc_vales, indent=2))
        # revenues = sheet.get_nwc_ratios()
        # print("Revenues:", json.dumps(revenues, indent=2))
        # nwc_values = sheet.get_nwc_values()
        # nwc_values = sheet.sort_dict_by_key(nwc_values)
        # print("NWC Values:", json.dumps(nwc_values, indent=2))

        free_cash_flow = Counter(no_pat) - \
          Counter(capital_expanditure) + \
          Counter(nwc_investment) + \
          Counter(DA)
        free_cash_flow = sheet.sort_dict_by_key(free_cash_flow)
        print("Free Cash Flow:", json.dumps(free_cash_flow, indent=2))
    else:
        print("Sheet not found.")

EBIT Projections: {
  "2025": 12499,
  "2026": 13348.6095,
  "2027": 14354.51,
  "2028": 15591.5,
  "2029": 17489,
  "2030": 18991.933625,
  "2031": 19656.651301874997,
  "2032": 20049.784327912497,
  "2033": 20450.780014470743,
  "2034": 20859.79561476016
}
D&A Projections: {
  "2025": 2124,
  "2026": 2206.937,
  "2027": 2552.8105699999996,
  "2028": 4153.5,
  "2029": 3658,
  "2030": 3932.35,
  "2031": 4069.982249999999,
  "2032": 4151.3818949999995,
  "2033": 4234.409532899999,
  "2034": 4319.097723557999
}
Tax Rate: 0.2
Projected Format Taxes: {
  "2025": 2499.8,
  "2026": 2669.7219000000005,
  "2027": 2870.902,
  "2028": 3118.3,
  "2029": 3497.8,
  "2030": 3798.3867250000003,
  "2031": 3931.330260375,
  "2032": 4009.9568655824996,
  "2033": 4090.156002894149,
  "2034": 4171.959122952033
}
No PAT: {
  "2025": 9999.2,
  "2026": 10678.8876,
  "2027": 11483.608,
  "2028": 12473.2,
  "2029": 13991.2,
  "2030": 15193.546900000001,
  "2031": 15725.321041499998,
  "2032": 16039.82746232999

In [ ]:
# import json
# import re

# def col_to_num(col_str):
#     """Convert column letter string (e.g., 'C', 'AA') to 1-based number."""
#     num = 0
#     for c in col_str.upper():
#         num = num * 26 + (ord(c) - ord('A')) + 1
#     return num

# def num_to_col(col_num):
#     """Convert 1-based column number to letter string (e.g., 3 to 'C')."""
#     string = ""
#     while col_num > 0:
#         col_num, remainder = divmod(col_num - 1, 26)
#         string = chr(65 + remainder) + string
#     return string

# def get_cell_id_str(col_num, row_num):
#      """Create cell identifier string (e.g., 'C7') from column and row numbers."""
#      return f"{num_to_col(col_num)}{row_num}"

# def parse_range(range_str):
#     """Parse a range string like 'C7:O25' into numerical coordinates."""
#     parts = range_str.split(':')
#     if len(parts) != 2:
#         return None # Invalid format
#     start_cell_str, end_cell_str = parts

#     # Use regex to separate column letters and row numbers
#     start_match = re.match(r'([A-Z]+)(\d+)', start_cell_str.upper())
#     end_match = re.match(r'([A-Z]+)(\d+)', end_cell_str.upper())

#     if not start_match or not end_match:
#         return None # Invalid cell format

#     start_col_str, start_row_str = start_match.groups()
#     end_col_str, end_row_str = end_match.groups()

#     try:
#         start_row = int(start_row_str)
#         end_row = int(end_row_str)
#         start_col = col_to_num(start_col_str)
#         end_col = col_to_num(end_col_str)
#         # Ensure start is less than or equal to end for iteration
#         start_row, end_row = sorted((start_row, end_row))
#         start_col, end_col = sorted((start_col, end_col))
#         return start_col, start_row, end_col, end_row
#     except ValueError:
#         return None # Invalid row number (not an integer)

# def is_numerical(value):
#     """Check if a value is a Python int or float."""
#     return isinstance(value, (int, float))


# def load_json_from_file(file_path):
#   try:
#       with open(file_path, 'r') as f:
#           json_data = json.load(f)
#   except FileNotFoundError:
#       print(f"Error: File not found at {file_path}")
#       exit()
#   except json.JSONDecodeError:
#       print(f"Error: Could not decode JSON from {file_path}")
#       exit()
#   except Exception as e:
#       print(f"An unexpected error occurred during file reading: {e}")
#       exit()
#   finally:
#       return json_data

# # try:
# #     with open(file_path, 'r') as f:
# #         json_data = json.load(f)
# # except FileNotFoundError:
# #     print(f"Error: File not found at {file_path}")
# #     exit()
# # except json.JSONDecodeError:
# #     print(f"Error: Could not decode JSON from {file_path}")
# #     exit()
# # except Exception as e:
# #     print(f"An unexpected error occurred during file reading: {e}")
# #     exit()

# # Replace 'response.json' with the actual path to your file
# file_path = 'response.json'

# json_data = load_json_from_file(file_path)

# result_dict = {}

# try:
#     # Navigate down the JSON structure to the sheets data
#     sheets = json_data.get("data", {}).get("model", {}).get("workbook", {}).get("sheets", {})

#     # Iterate through each sheet in the workbook
#     for sheet_name, sheet_data in sheets.items():
#         cells = sheet_data.get("cells", {})
#         sheet_sections_cells = set() # Set to store all cell IDs within sections for the current sheet

#         # First pass over the current sheet's cells: Identify all defined section ranges
#         for cell_id, cell_props in cells.items():
#             section_info = cell_props.get("section")
#             if section_info:
#                 # Section info is typically in the format "section_name;start_cell:end_cell"
#                 parts = section_info.split(';')
#                 if len(parts) == 2:
#                     section_range_str = parts[1]
#                     range_coords = parse_range(section_range_str)

#                     if range_coords:
#                         start_col, start_row, end_col, end_row = range_coords

#                         # Add all cell identifiers within this range to the set
#                         for r in range(start_row, end_row + 1):
#                             for c in range(start_col, end_col + 1):
#                                 cell_identifier = get_cell_id_str(c, r)
#                                 sheet_sections_cells.add(cell_identifier)
#                     # else:
#                     #     print(f"Warning: Could not parse range '{section_range_str}' for section '{parts[0]}' in cell {cell_id} on sheet '{sheet_name}'")


#         # Second pass over the identified section cells: Extract numerical 'computed' values
#         for cell_id in sheet_sections_cells:
#              # Get the cell properties for this cell ID from the current sheet
#              cell_props = cells.get(cell_id)

#              if cell_props:
#                  computed_value = cell_props.get("value")

#                  # Check if the computed value is a number (int or float)
#                  if is_numerical(computed_value):
#                       # Add to the result dictionary in the format cell : num_value
#                       # Note: If the same cell ID (e.g., "A1") is in sections on different sheets,
#                       # this will store the value from the last sheet processed.
#                       result_dict[cell_id] = computed_value
#              # else:
#              #     print(f"Warning: Cell ID '{cell_id}' derived from a section range was not found in the cell data for sheet '{sheet_name}'")


# except Exception as e:
#     print(f"An error occurred while processing the data: {e}")
    # You might want more specific error handling based on potential issues in your data

# Print the resulting dictionary as JSON
# print(json.dumps(result_dict, indent=4))
# result_dict

In [ ]:
# json_data = load_json_from_file(file_path)

# ebit_projections = {}

# try:
#     # Navigate down the JSON structure to the sheets data
#     sheets = json_data.get("data", {}).get("model", {}).get("workbook", {}).get("sheets", {})

#     # We specifically want the data from the "10 Year DCF - Growth Exit" sheet
#     sheet_name = "10 Year DCF - Growth Exit"
#     sheet_data = sheets.get(sheet_name)

#     if not sheet_data:
#         print(f"Error: Sheet '{sheet_name}' not found in the JSON data.")
#         exit()

#     cells = sheet_data.get("cells", {})

#     # Define the row number and the start/end columns for the EBIT projection data
#     target_row = 381
#     start_col_letter = 'E'
#     end_col_letter = 'N'

#     start_col_num = col_to_num(start_col_letter)
#     end_col_num = col_to_num(end_col_letter)

#     # The first year (2025) corresponds to the start column (E)
#     start_year = 2025

#     # Iterate through the relevant columns
#     for col_num in range(start_col_num, end_col_num + 1):
#         col_letter = num_to_col(col_num)
#         cell_id = f"{col_letter}{target_row}"

#         # Calculate the year based on the column position relative to the start column
#         current_year = start_year + (col_num - start_col_num)

#         cell_data = cells.get(cell_id)

#         # Extract the computed value if the cell exists and add to the dictionary
#         if cell_data and "computed" in cell_data:
#              ebit_projections[current_year] = cell_data["value"]
#         # Optional: handle cases where the cell or computed value is missing
#         # else:
#         #     ebit_projections[current_year] = None # Or some other indicator


# except Exception as e:
#     print(f"An error occurred while processing the data: {e}")

# # Print the resulting dictionary as JSON
# print(json.dumps(ebit_projections, indent=4))